## Handle Imports

In [72]:
import numpy as np
import pandas as pd
import heapq as pq
import csv
import statistics as stats
import copy
import math
import time
import matplotlib.pyplot as plt

## Load Up methods

### Load Countries

In [73]:
def load_countries():
    countries = {}

    with open("countries.csv") as file:
        reader = csv.DictReader(file)
        for row in reader:
            key = row["Country"]
            countries[key] = {name: float(value) for name, value in row.items() if name != "Country"}

    return countries

# LOAD Methods

In [74]:
def load_up():
    
    #load and create df
    countryList = load_countries()
    root_country_df = pd.DataFrame(countryList).transpose()
    
    return root_country_df

In [75]:
def get_country_names(df):
    return df.index.to_list()

### Jupyter Run

In [76]:
country_df = load_up()
root_country_df = load_up()
COUNTRY_NAMES = get_country_names(country_df)

In [77]:
COUNTRY_NAMES

['Atlantis', 'Dinotopoia', 'Erewhon', "King's Landing", 'The Vale']

## State Quality Function

# Set Maslow Constants

In [78]:
#different resource levels
L_ONE_RESOURCES = {'food': 1, 'water': 1}
L_TWO_RESOURCES = {'housing': 1, 'timber': 1, 'metallic alloy': 0.5, 'electronics': 2, 'potential fossil energy': 1} 
L_THREE_RESOURCES = {'community buildings': 0.05, 'jobs': 1, 'high school education': 1, 'college education': 1, 'universities': 1, 'marriages': 1} 
L_FOUR_RESOURCES = {'children': 2.5, 'renewable energy': 1}
L_FIVE_RESOURCES = {'food waste': -1, 'water waste': -1, 'land waste': -1, 'timber waste': -1, 'nobel prizes': 0.02}

In [79]:
#list of levels
LEV_LIST = [L_ONE_RESOURCES, L_TWO_RESOURCES, L_THREE_RESOURCES, L_FOUR_RESOURCES, L_FIVE_RESOURCES]

## Level Function

In [80]:
def leveldf(df, country, level, level_function_accessor):
    
    levelSat = False
    marUtility = False
    wasteRate = False
    mult = []
    average = 0

    for key, value in level_function_accessor[level-1].items():
        countryVal = df.loc[country, key]
        mult.append(countryVal/value)
        if countryVal < value:
            levelSat = True
            
        #marginal utility
        if countryVal > value*3:
            marUtility = True
            
            if countryVal > value * 6:
                wasteRate = True
                
    
    average = stats.mean(mult)
    
    if levelSat:
        average = average*0.01
        
    if marUtility:
        average = average*0.01
        
    if wasteRate:
        average = average*0.00001
        
    return average

### Level Function Unit Test

In [81]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste,br,dr
Atlantis,20.0,30.0,50.0,30.0,20.0,40.0,34.0,0.0,10.0,500.0,...,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0,5.0,3.0
Dinotopoia,15.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0,2.0,4.0
Erewhon,21.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0,5.0,1.0
King's Landing,80.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0,10.0,2.0
The Vale,43.0,100.0,50.0,100.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0,3.0,3.0


In [82]:
num = leveldf(country_df, 'Atlantis', 1, LEV_LIST)
num

4.000000000000001e-06

In [83]:
num = leveldf(country_df, 'Atlantis', 3, LEV_LIST)
num

6.810000000000002e-05

In [84]:
num = leveldf(country_df, 'Dinotopoia', 4, LEV_LIST)
num

1.0140000000000001e-05

In [85]:
num = leveldf(country_df, 'Atlantis', 5, LEV_LIST)
num

3.34e-08

Population Function

In [86]:
def pop(df):
    for row in range(len(df)):
        values = df.iloc[row]
        popVal = values[0]
        BR = round(np.random.normal(loc=values[len(values) - 2], scale=1.0, size=None))
        DR = round(np.random.normal(loc=values[len(values) - 1], scale=1.0, size=None))
        df.iat[row, 0] = popVal + BR - DR
        if df.iat[row, 0] < 0:
            df.iat[row, 0] = 1

In [87]:
pop(country_df)
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste,br,dr
Atlantis,22.0,30.0,50.0,30.0,20.0,40.0,34.0,0.0,10.0,500.0,...,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0,5.0,3.0
Dinotopoia,14.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0,2.0,4.0
Erewhon,25.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0,5.0,1.0
King's Landing,87.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0,10.0,2.0
The Vale,42.0,100.0,50.0,100.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0,3.0,3.0


## Maslow Function

In [88]:
def maslowLevelVals(df, country, level, level_function_accessor):
    
    maslowList = []
    ## pop(df)
    ## normalize
    norm_df = df.copy()
    
    for row in range(len(norm_df)):
        
        values = norm_df.iloc[row]
        popVal = values[0]
        
        for vals in range(1, len(values)):
            values[vals] = values[vals]/popVal
    
    # maslow function
    for num in range(1, level+1):
        levValue = leveldf(norm_df, country, num, level_function_accessor)
        maslowList.append(levValue)
            
    return maslowList

In [89]:
maslowL = maslowLevelVals(country_df, 'Atlantis', 5, LEV_LIST)
maslowL

[1.8181818181818183,
 5.136363636363636e-09,
 3.095454545454545e-08,
 0.0002363636363636364,
 1.5181818181818183e-07]

In [90]:
def maslowList(lst):
    sumVal = 0
    sumVal = sumVal + lst[0] * 0.10
    sumVal = sumVal + lst[1] * 10
    sumVal = sumVal + lst[2] * 100
    sumVal = sumVal + lst[3] * 1000
    return sumVal

In [91]:
masVal = maslowList(maslowL)
masVal

0.4181849650000001

# Maslow

In [92]:
def maslowVal(df, country, level, LEV_LIST):
    
    lst = maslowLevelVals(df, country, level, LEV_LIST)
    val = maslowList(lst)
    return val

## Maslow Function Test Case

In [93]:
num = maslowVal(country_df, 'The Vale', 5, LEV_LIST)
num

10.654770739285713

In [94]:
num = maslowVal(country_df, 'Atlantis', 5, LEV_LIST)
num

0.4181849650000001

In [95]:
num = maslowVal(country_df, 'Dinotopoia', 5, LEV_LIST)
num

0.004134007142857143

## Transfer Operation

### Transform Templates

In [96]:
HOUSING_TM = ['HOUSING', {'land': 1, 'population': 5, 'water': 5, 'metallic element': 1, 'timber': 5, 'metallic alloy': 3, 'potential fossil usable': 5}, {'housing': 1, 'housing waste': 1, 'timber waste': 1, 'population': 5, 'water': 4}]
ALLOYS_TM = ['ALLOYS', {'population': 1, 'metallic element': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'metallic alloy': 1, 'metallicAlloy waste': 1, 'water': 2}]
ELECTRONICS_TM = ['ELECTRONICS', {'population': 1, 'metallic element': 3, 'metallic alloy': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'electronics': 2, 'electronics waste': 1, 'water': 2}]
FARM_TM = ['FARM', {'population': 1, 'land' : 1, 'water': 3}, {'food': 5, 'population': 1}]
LOGGING_TM = ['LOGGING', {'population': 3, 'potential fossil usable': 3}, {'population': 3, 'timber': 5}]
PURIFY_WATER_TM = ['PURIFY_WATER', {'population': 3, 'potential fossil usable': 3}, {'population': 3, 'water': 5}]
FOSSIL_ENERGY_TM = ['FOSSIL_ENERGY', {'population': 5, 'potential fossil energy': 2}, {'population': 5, 'potential fossil usable': 1, 'potential fossil energy waste': 1}]
RENEWABLE_ENERGY_TM = ['RENEWABLE_ENERGY', {'population': 5, 'potential fossil usable': 3}, {'population': 5, 'renewable energy': 1, 'renewable energy waste': 1}]
COMMUNITY_BUILDING_TM = ['COMMUNITY_BUILDING', {'land': 1, 'population': 10, 'water': 5, 'metallic element': 3, 'timber': 8, 'metallic alloy': 5, 'potential fossil usable': 5}, {'community buildings': 1, 'housing waste': 1, 'timber waste': 1, 'metallicAlloy waste': 1, 'population': 10, 'water': 4}]
UNIVERSITY_TM = ['UNIVERSITY', {'land': 1, 'population': 50, 'water': 5, 'metallic element': 5, 'timber': 10, 'metallic alloy': 5, 'potential fossil usable': 5}, {'universities': 1, 'population': 50, 'water': 3, 'timber waste': 1, 'metallicAlloy waste': 1}]
JOB_HS_TM = ['JOB_HS', {'population': 25, 'high school education': 1}, {'population': 25, 'jobs': 1}]
JOB_C_TM = ['JOB_C', {'population': 50, 'college education': 1}, {'population': 50, 'jobs': 1}]
HIGHSCHOOL_ED_TM = ['HIGHSCHOOL_ED', {'population': 15, 'housing': 1, 'children': 1}, {'population': 16, 'housing': 1, 'high school education': 1}]
COLLEGE_ED_TM = ['COLLEGE_ED', {'population': 50, 'housing': 1, 'universities': 1, 'high school education': 1}, {'population': 50, 'housing': 1, 'universities': 1, 'college education': 1}]
MARRIAGE_TM = ['MARRIAGE', {'population': 2, 'housing': 1}, {'population': 2, 'housing': 1, 'marriages': 1}]
CHILDREN_TM = ['CHILDREN', {'marriages': 1, 'housing': 1}, {'marriages': 1, 'housing': 1, 'children': 2}]
NOBEL_PRIZE_TM = ['NOBEL_PRIZE', {'population': 1, 'universities': 10, 'college education': 50, 'potential fossil usable': 10}, {'population': 1, 'universities': 10, 'college education': 50, 'nobel prizes': 1}]

ALL_TEMPLATES_TRANSFORM = [HOUSING_TM, ALLOYS_TM, ELECTRONICS_TM, FARM_TM, LOGGING_TM, PURIFY_WATER_TM, FOSSIL_ENERGY_TM, 
                          RENEWABLE_ENERGY_TM, COMMUNITY_BUILDING_TM, UNIVERSITY_TM, JOB_HS_TM, JOB_C_TM, HIGHSCHOOL_ED_TM,
                          COLLEGE_ED_TM, MARRIAGE_TM, CHILDREN_TM, NOBEL_PRIZE_TM]

### Transform Function

In [97]:
def transform(df, country, transform_template):
    
    allowed = True
    
    #check if transform is possible
    for key in transform_template[1]:
        val = transform_template[1][key]
        if(df.loc[country, key] - val < 0):
            allowed = False
    
    if(allowed):
        #remove input resoures
        for key in transform_template[1]:
            val = transform_template[1][key]
            df.loc[country, key] -= val

        #add output resources
        for key in transform_template[2]:
            val = transform_template[2][key]
            df.loc[country, key] += val
    return allowed

### Transform Unit Test

In [98]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste,br,dr
Atlantis,22.0,30.0,50.0,30.0,20.0,40.0,34.0,0.0,10.0,500.0,...,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0,5.0,3.0
Dinotopoia,14.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0,2.0,4.0
Erewhon,25.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0,5.0,1.0
King's Landing,87.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0,10.0,2.0
The Vale,42.0,100.0,50.0,100.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0,3.0,3.0


In [99]:
transform(country_df, 'Atlantis', FARM_TM)

True

In [100]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste,br,dr
Atlantis,22.0,35.0,47.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0,5.0,3.0
Dinotopoia,14.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0,2.0,4.0
Erewhon,25.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0,5.0,1.0
King's Landing,87.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0,10.0,2.0
The Vale,42.0,100.0,50.0,100.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0,3.0,3.0


In [101]:
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)
transform(country_df, 'The Vale', FARM_TM)

True

In [102]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste,br,dr
Atlantis,22.0,35.0,47.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0,5.0,3.0
Dinotopoia,14.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0,2.0,4.0
Erewhon,25.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0,5.0,1.0
King's Landing,87.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0,10.0,2.0
The Vale,42.0,130.0,32.0,94.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0,3.0,3.0


In [103]:
transform(country_df, 'The Vale', FARM_TM)

True

In [104]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste,br,dr
Atlantis,22.0,35.0,47.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0,5.0,3.0
Dinotopoia,14.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0,2.0,4.0
Erewhon,25.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0,5.0,1.0
King's Landing,87.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0,10.0,2.0
The Vale,42.0,135.0,29.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0,3.0,3.0


# Transfer Function Definition

### Transfer Templates

In [105]:
FOOD_TR = ['FOOD', {'food': 5}]
WATER_TR = ['WATER', {'water': 5}]
TIMBER_TR = ['TIMBER', {'timber': 5}]
METALLIC_ELEMENT_TR = ['METALLIC ELEMENT', {'metallic element':5}]
METALLIC_ALLOY_TR = ['METALLIC ALLOY', {'metallic alloy':5}]
ELECTRONICS_TR = ['ELECTRONICS', {'electronics':5}]
POTENTIAL_FOSSIL_ENERGY_TR = ['POTENTIAL_FOSSIL_ENERGY_TR',{'potential fossil energy':5}]
POTENTIAL_FOSSIL_USABLE_TR = ['POTENTIAL_FOSSIL_USABLE_TR',{'potential fossil usable':5}]
RENEWABLE_ENERGY_TR = ['RENEWABLE_ENERGY_TR',{'renewable energy':5}]

In [106]:
ALL_TEMPLATES_TRANSFER = [FOOD_TR,WATER_TR,TIMBER_TR,METALLIC_ELEMENT_TR,METALLIC_ALLOY_TR,ELECTRONICS_TR,
                          POTENTIAL_FOSSIL_ENERGY_TR,POTENTIAL_FOSSIL_USABLE_TR,RENEWABLE_ENERGY_TR]

### Transfer Function

In [107]:
def transfer(df, country1, country2, transfer_template):

    allowed = True

    #check for validity
    
    if country1==country2:
        allowed = False

    for key in transfer_template[1]:
        val = transfer_template[1][key]
        if (df.loc[country1,key] - val < 0):
            allowed = False
            
    if(allowed):
        #remove resource from country 1
        for key in transfer_template[1]:
            val = transfer_template[1][key]
            df.loc[country1, key] -= val

        #add resource to country 2
        for key in transfer_template[1]:
            val = transfer_template[1][key]
            df.loc[country2, key] += val
            
    return allowed

## Transfer Function Test Case

In [108]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste,br,dr
Atlantis,22.0,35.0,47.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0,5.0,3.0
Dinotopoia,14.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0,2.0,4.0
Erewhon,25.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0,5.0,1.0
King's Landing,87.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0,10.0,2.0
The Vale,42.0,135.0,29.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0,3.0,3.0


In [109]:
transfer(country_df, 'Atlantis', 'The Vale', WATER_TR)

True

In [110]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste,br,dr
Atlantis,22.0,35.0,42.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0,5.0,3.0
Dinotopoia,14.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0,2.0,4.0
Erewhon,25.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0,5.0,1.0
King's Landing,87.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0,10.0,2.0
The Vale,42.0,135.0,34.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0,3.0,3.0


In [111]:
transfer(country_df, 'The Vale', 'Atlantis', WATER_TR)

True

In [112]:
transfer(country_df, 'The Vale', 'Atlantis', WATER_TR)

True

In [113]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste,br,dr
Atlantis,22.0,35.0,52.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0,5.0,3.0
Dinotopoia,14.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0,2.0,4.0
Erewhon,25.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0,5.0,1.0
King's Landing,87.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0,10.0,2.0
The Vale,42.0,135.0,24.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0,3.0,3.0


# Trade Function Definition

In [114]:
def trade(df, country1, country2, transfer1, transfer2):
    # A trade deal between two countries country1 and country2
    # for which country uses transfer1 and country2 uses transfer2
    if transfer1 == transfer2:
        return False
    if transfer(df, country1, country2, transfer1):
        if not transfer(df, country2, country1, transfer2):
            transfer(df, country2, country1, transfer1)
            return False
        else:
            return True
    else:
        return False

## Trade Function Test Case

In [115]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste,br,dr
Atlantis,22.0,35.0,52.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0,5.0,3.0
Dinotopoia,14.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0,2.0,4.0
Erewhon,25.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0,5.0,1.0
King's Landing,87.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0,10.0,2.0
The Vale,42.0,135.0,24.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0,3.0,3.0


In [116]:
trade(country_df,'Atlantis','The Vale',FOOD_TR, WATER_TR)

True

In [117]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste,br,dr
Atlantis,22.0,30.0,57.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0,5.0,3.0
Dinotopoia,14.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0,2.0,4.0
Erewhon,25.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0,5.0,1.0
King's Landing,87.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0,10.0,2.0
The Vale,42.0,140.0,19.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0,3.0,3.0


In [118]:
trade(country_df,'Atlantis','The Vale', WATER_TR, FOOD_TR)

True

In [119]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste,br,dr
Atlantis,22.0,35.0,52.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0,5.0,3.0
Dinotopoia,14.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0,2.0,4.0
Erewhon,25.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0,5.0,1.0
King's Landing,87.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0,10.0,2.0
The Vale,42.0,135.0,24.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0,3.0,3.0


# Node Definition

In [120]:
class Node:
    def __init__(self, state_quality, exp_util, sched, countrydf):
        self.state_quality = state_quality
        self.exp_util = exp_util
        self.sched = sched
        self.countrydf = countrydf
    def getSqual(self):
        return self.state_quality
    def getExutil(self):
        return self.exp_util
    def getSched(self):
        return self.sched
    def getCountrydf(self):
        return self.countrydf
    def __lt__(self, other):
        return self.exp_util < other.exp_util
    def __str__(self):
        s = "Node with State Quality: {}\nExpected Utility: {}\nSchedule: {}\nDataframe:\n{}".format(self.state_quality, self.exp_util, self.sched, self.countrydf)
        return s

# Bounded Priority Queue Definition

In [121]:
class BoundedPriorityQueue:
    # can change bound
    def __init__(self, queue=list(), bound=5):
        self.bound = bound
        self.queue = queue
    def push(self, priority, item):
        if len(self.queue) == self.bound:
            pq.heappushpop(self.queue, (priority, item))
        else:
            pq.heappush(self.queue, (priority, item))
    def pop(self):
        if len(self.queue) >= 1:
            return pq.heappop(self.queue)[-1]
        print("no nodes on fringe")
    ##maybe delete    
    def items(self):
        return list(item for _,_, item in self.queue)
    def clear(self):
        self.queue.clear()
    def len(self):
        return len(self.queue)
    def getState(self, index):
        if len(self.queue) > index:
            return self.queue[index][1]
        print("index out of bound")
    def pState(self, index):
        if len(self.queue) > index:
            print(self.queue[index][1])
        print("index out of bound")
    def getPriority(self, index):
        if len(self.queue) > index:
            return self.queue[index][0]
        print("index out of bound")
    def popBest(self):
        if len(self.queue) >= 1:
            return pq.nlargest(1, self.queue)[0][1]
        print("no nodes on fringe")
    def copy(self):
        return BoundedPriorityQueue(self.queue.copy())

## Generate Succcessors

In [122]:
def generate_successors(parentNode, my_country, LEV_LIST):
    
    successors = BoundedPriorityQueue()
    prevSched = parentNode.getSched()
    
    for resource in ALL_TEMPLATES_TRANSFORM:
        
        new_df = parentNode.getCountrydf().copy()
        
        if transform(new_df, my_country, resource):
            state_qual = maslowVal(new_df, my_country, 5, LEV_LIST)
            new_schedule = prevSched.copy()
            new_state = Node(state_qual, 0, new_schedule, new_df)
            new_state.sched.append("TRANSFORM: " + resource[0])
            new_state.exp_util = expected_utility_transform(new_state, my_country, LEV_LIST)
            successors.push(new_state.exp_util, new_state)


    for resource_1 in ALL_TEMPLATES_TRANSFER:
        for resource_2 in ALL_TEMPLATES_TRANSFER:
            for other_country in COUNTRY_NAMES:
                
                new_df = parentNode.getCountrydf().copy()
                
                if trade(new_df, my_country, other_country, resource_1, resource_2):
                    state_qual = maslowVal(new_df, my_country, 5, LEV_LIST)
                    state_qual_other = maslowVal(new_df, other_country, 5, LEV_LIST)
                    new_schedule = prevSched.copy()
                    new_state = Node(state_qual, 0, new_schedule, new_df)
                    trade_name = "TRADE: " + my_country + " GIVES " + resource_1[0] + ", " + other_country + " GIVES " + resource_2[0]
                    new_state.sched.append(trade_name)
                    new_state.exp_util = expected_utility_trade(L, k, x_0, new_state, my_country, LEV_LIST, other_country)
                    successors.push(new_state.exp_util, new_state)
                    
    return successors

# Reward Functions

In [123]:
def undiscounted_reward(node, my_country,LEV_LIST):
    root_node_score = maslowVal(root_country_df, my_country, 5, LEV_LIST)
    cur_node_score = maslowVal(node.getCountrydf(), my_country, 5, LEV_LIST)
    return cur_node_score - root_node_score

In [124]:
GAMMA = 0.05

def discounted_reward(node, my_country, LEV_LIST):
    return (GAMMA ** len(node.getSched()))*undiscounted_reward(node, my_country, LEV_LIST)

# Probability function

In [125]:
L = 1.0
k = 1.0
x_0 = 0.0

def probability(L, k, node, other_country, LEV_LIST, x_0):
    return (L/(1+math.exp(-k*(discounted_reward(node, other_country, LEV_LIST)-x_0))))

In [126]:
COST_FAILURE = -0.05

def expected_utility_trade(L, k, x_0, node, my_country, LEV_LIST, other_country):
    prob = probability(L, k, node, other_country, LEV_LIST, x_0)
    success = prob * discounted_reward(node, other_country, LEV_LIST)
    fail = (1 - prob) * COST_FAILURE
    other_country_utility = success + fail
    prob = probability(L, k, node, my_country, LEV_LIST, x_0)
    success = prob * discounted_reward(node, my_country, LEV_LIST)
    fail = (1 - prob) * COST_FAILURE
    my_country_utility = success + fail
    total = (other_country_utility+my_country_utility)/2
    return total

In [127]:
PROB = 0.95
COST_FAILURE = -0.05
def expected_utility_transform(node, my_country, LEV_LIST):
    success = PROB * discounted_reward(node, my_country, LEV_LIST)
    fail = (1-PROB)*COST_FAILURE 
    return success + fail

# Node Test Cases 

In [128]:
country_df

,population,food,water,land,timber,housing,metallic element,metallic alloy,electronics,potential fossil energy,...,water waste,land waste,timber waste,housing waste,metallicAlloy waste,electronics waste,potential fossil energy waste,renewable energy waste,br,dr
Atlantis,22.0,35.0,52.0,29.0,20.0,40.0,34.0,0.0,10.0,500.0,...,0.0,-2.0,-10.0,-5.0,-10.0,-14.0,-20.0,-1.0,5.0,3.0
Dinotopoia,14.0,35.0,80.0,100.0,40.0,230.0,20.0,25.0,30.0,1000.0,...,0.0,-23.0,-20.0,-20.0,-20.0,-23.0,-20.0,-5.0,2.0,4.0
Erewhon,25.0,40.0,60.0,20.0,76.0,100.0,30.0,120.0,75.0,2000.0,...,0.0,-30.0,-20.0,-10.0,-30.0,-20.0,-10.0,-3.0,5.0,1.0
King's Landing,87.0,200.0,100.0,50.0,40.0,80.0,100.0,10.0,20.0,2000.0,...,-120.0,-10.0,-20.0,-23.0,-10.0,-10.0,-25.0,-20.0,10.0,2.0
The Vale,42.0,135.0,24.0,93.0,110.0,70.0,150.0,100.0,100.0,1000.0,...,0.0,-40.0,-45.0,-50.0,-10.0,-20.0,-10.0,-18.0,3.0,3.0


In [129]:
state_qual = 1.2
exp_util = 3.0
sched = ['ROOT STATE']
test_node = Node(state_qual, exp_util, sched, country_df)

In [130]:
expected_utility_transform(test_node, 'Atlantis', LEV_LIST)

-0.0037306967655681817

In [131]:
successors = generate_successors(test_node, 'Atlantis', LEV_LIST)

In [132]:
print(successors.len())
suc1 = successors.pop()

5


In [133]:
yes = BoundedPriorityQueue()
yes.push(1, test_node)
yes.push(1, test_node)
print(yes.len())
print(yes.pop())

5
Node with State Quality: 0.4363667831818182
Expected Utility: -0.0025561371110056842
Schedule: ['ROOT STATE', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              22.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            14.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               25.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        87.0  200.0  100.0   50.0    40.0     80.0   
The Vale              42.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing             100.0            10.0         20.0   
The Vale                   150.0           100.0        100.0   

                potential fossil energy  ...  

In [134]:
print(suc1.getSched())

['ROOT STATE', 'TRANSFORM: MARRIAGE']


In [135]:
successors.getPriority(1)

-0.0025345462019147748

## Search Function

In [136]:
def search(root_node, my_country,level_accessor, max_iter):
    
    num = 1
    
    fringe = generate_successors(root_node, my_country, level_accessor).copy()
    newfringe = BoundedPriorityQueue()
    for i in range(1, max_iter):
        fringe_bound = fringe.len()
        for j in range(0,fringe_bound):
            successors = generate_successors(fringe.pop(), my_country, level_accessor)
            for k in range(0,successors.len()-1):
                newfringe.push(successors.getState(k).getExutil(),successors.getState(k))
                num = num + 1
        fringe = newfringe.copy()
        newfringe.clear()
    return fringe.popBest()

### Search Function Test Cases

In [137]:
""" seconds = []
depth = []
for x in range(2, 6):
    print(x)
    sum = 0
    for y in range(1, 3):
        initial = time.time()
        nodeBest = search(test_node, 'Atlantis', LEV_LIST, x)
        sum += time.time() - initial
    sum = sum / 3
    seconds.append(sum)
    depth.append(x)
plt.plot(depth, seconds, color='red', marker='o')
plt.title('Average Runtime Vs Depth', fontsize=14)
plt.xlabel('Depth', fontsize=14)
plt.ylabel('Runtime', fontsize=14)
plt.grid(True)
plt.show() """

" seconds = []\ndepth = []\nfor x in range(2, 6):\n    print(x)\n    sum = 0\n    for y in range(1, 3):\n        initial = time.time()\n        nodeBest = search(test_node, 'Atlantis', LEV_LIST, x)\n        sum += time.time() - initial\n    sum = sum / 3\n    seconds.append(sum)\n    depth.append(x)\nplt.plot(depth, seconds, color='red', marker='o')\nplt.title('Average Runtime Vs Depth', fontsize=14)\nplt.xlabel('Depth', fontsize=14)\nplt.ylabel('Runtime', fontsize=14)\nplt.grid(True)\nplt.show() "

In [138]:
"""print(nodeBest)"""

'print(nodeBest)'

In [139]:
"""
seconds = [52.52, 79.73,99.55,115.01]
bound = [2,3,4,5]
plt.plot(bound, seconds, color='red', marker='o')
plt.title('Runtime Vs Queue Bound', fontsize=14)
plt.xlabel('Queue Bound', fontsize=14)
plt.ylabel('Runtime', fontsize=14)
plt.grid(True)
plt.show()
"""

"\nseconds = [52.52, 79.73,99.55,115.01]\nbound = [2,3,4,5]\nplt.plot(bound, seconds, color='red', marker='o')\nplt.title('Runtime Vs Queue Bound', fontsize=14)\nplt.xlabel('Queue Bound', fontsize=14)\nplt.ylabel('Runtime', fontsize=14)\nplt.grid(True)\nplt.show()\n"

In [140]:
initial = time.time()
nodeBest = search(test_node, 'The Vale', LEV_LIST, 3)
print(time.time() - initial)

199.0187704563141


In [141]:
print(nodeBest)

Node with State Quality: 10.595265858333333
Expected Utility: -0.002052833847940896
Schedule: ['ROOT STATE', 'TRANSFORM: RENEWABLE_ENERGY']
Dataframe:
                population   food  water   land  timber  housing  \
Atlantis              22.0   35.0   52.0   29.0    20.0     40.0   
Dinotopoia            14.0   35.0   80.0  100.0    40.0    230.0   
Erewhon               25.0   40.0   60.0   20.0    76.0    100.0   
King's Landing        87.0  200.0  100.0   50.0    40.0     80.0   
The Vale              42.0  135.0   24.0   93.0   110.0     70.0   

                metallic element  metallic alloy  electronics  \
Atlantis                    34.0             0.0         10.0   
Dinotopoia                  20.0            25.0         30.0   
Erewhon                     30.0           120.0         75.0   
King's Landing             100.0            10.0         20.0   
The Vale                   150.0           100.0        100.0   

                potential fossil energy  ...  wat

In [ ]:
nodeBest = search(test_node, 'Erewhon', LEV_LIST, 10)

In [ ]:
print(nodeBest)

In [ ]:
nodeBest = search(test_node, 'Dinotopoia', LEV_LIST, 10)

In [ ]:
print(nodeBest)

In [ ]:
nodeBest = search(test_node, 'The Vale', LEV_LIST, 10)

In [ ]:
print(nodeBest)

In [ ]:
nodeBest = search(test_node, 'The Vale', LEV_LIST, 100)

In [ ]:
print(nodeBest)

# Driver

In [ ]:
def driver(country, tree_size):
    #load in countries
    country_df = load_up()
    COUNTRY_NAMES = get_country_names(country_df)
    
    #different resource levels
    L_ONE_RESOURCES = {'food': 1, 'water': 1}
    L_TWO_RESOURCES = {'housing': 1, 'timber': 1, 'metallic alloy': 0.5, 'electronics': 3, 'potential fossil energy': 1} 
    L_THREE_RESOURCES = {'community buildings': 0.05, 'jobs': 1, 'high school education': 1, 'college education': 1, 'universities': 1, 'marriages': 1} 
    L_FOUR_RESOURCES = {'children': 2.5, 'renewable energy': 1}
    L_FIVE_RESOURCES = {'food waste': -1, 'water waste': -1, 'land waste': -1, 'timber waste': -1, 'nobel prizes': 0.02}
    
    LEV_LIST = [LEVELONERESOURCES, L_TWO_RESOURCES, L_THREE_RESOURCES, L_FOUR_RESOURCES, L_FIVE_RESOURCES]
    
    #transfer templates
    HOUSING_TM = ['HOUSING', {'land': 1, 'population': 5, 'water': 5, 'metallic element': 1, 'timber': 5, 'metallic alloy': 3, 'potential fossil usable': 5}, {'housing': 1, 'housing waste': 1, 'timber waste': 1, 'population': 5, 'water': 4}]
    ALLOYS_TM = ['ALLOYS', {'population': 1, 'metallic element': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'metallic alloy': 1, 'metallicAlloy waste': 1, 'water': 2}]
    ELECTRONICS_TM = ['ELECTRONICS', {'population': 1, 'metallic element': 3, 'metallic alloy': 2, 'water': 3, 'potential fossil usable': 3}, {'population': 1, 'electronics': 2, 'electronics waste': 1, 'water': 2}]
    FARM_TM = ['FARM', {'population': 1, 'land' : 1, 'water': 3}, {'food': 5, 'population': 1}]
    LOGGING_TM = ['LOGGING', {'population': 3, 'potential fossil usable': 3}, {'population': 3, 'timber': 5}]
    PURIFY_WATER_TM = ['PURIFY_WATER', {'population': 3, 'potential fossil usable': 3}, {'population': 3, 'water': 5}]
    FOSSIL_ENERGY_TM = ['FOSSIL_ENERGY', {'population': 5, 'potential fossil energy': 2}, {'population': 5, 'potential fossil usable': 1, 'potential fossil energy waste': 1}]
    RENEWABLE_ENERGY_TM = ['RENEWABLE_ENERGY', {'population': 5, 'potential fossil usable': 3}, {'population': 5, 'renewable energy': 1, 'renewable energy waste': 1}]
    COMMUNITY_BUILDING_TM = ['COMMUNITY_BUILDING', {'land': 1, 'population': 10, 'water': 5, 'metallic element': 3, 'timber': 8, 'metallic alloy': 5, 'potential fossil usable': 5}, {'community buildings': 1, 'housing waste': 1, 'timber waste': 1, 'metallicAlloy waste': 1, 'population': 10, 'water': 4}]
    UNIVERSITY_TM = ['UNIVERSITY', {'land': 1, 'population': 50, 'water': 5, 'metallic element': 5, 'timber': 10, 'metallic alloy': 5, 'potential fossil usable': 5}, {'universities': 1, 'population': 50, 'water': 3, 'timber waste': 1, 'metallicAlloy waste': 1}]
    JOB_HS_TM = ['JOB_HS', {'population': 25, 'high school education': 1}, {'population': 25, 'jobs': 1}]
    JOB_C_TM = ['JOB_C', {'population': 50, 'college education': 1}, {'population': 50, 'jobs': 1}]
    HIGHSCHOOL_ED_TM = ['HIGHSCHOOL_ED', {'population': 15, 'housing': 1, 'children': 1}, {'population': 16, 'housing': 1, 'high school education': 1}]
    COLLEGE_ED_TM = ['COLLEGE_ED', {'population': 50, 'housing': 1, 'universities': 1, 'high school education': 1}, {'population': 50, 'housing': 1, 'universities': 1, 'college education': 1}]
    MARRIAGE_TM = ['MARRIAGE', {'population': 2, 'housing': 1}, {'population': 2, 'housing': 1, 'marriages': 1}]
    CHILDREN_TM = ['CHILDREN', {'marriages': 1, 'housing': 1}, {'marriages': 1, 'housing': 1, 'children': 2}]
    NOBEL_PRIZE_TM = ['NOBEL_PRIZE', {'population': 1, 'universities': 10, 'college education': 50, 'potential fossil usable': 10}, {'population': 1, 'universities': 10, 'college education': 50, 'nobel prizes': 1}]

    #total transform template list
    ALL_TEMPLATES_TRANSFORM = [HOUSING_TM, ALLOYS_TM, ELECTRONICS_TM, FARM_TM, LOGGING_TM, PURIFY_WATER_TM, FOSSIL_ENERGY_TM, 
                          RENEWABLE_ENERGY_TM, COMMUNITY_BUILDING_TM, UNIVERSITY_TM, JOB_HS_TM, JOB_C_TM, HIGHSCHOOL_ED_TM,
                          COLLEGE_ED_TM, MARRIAGE_TM, CHILDREN_TM, NOBEL_PRIZE_TM]
    
    #transfer templates
    FOOD_TR = ['FOOD', {'food': 5}]
    WATER_TR = ['WATER', {'water': 5}]
    TIMBER_TR = ['TIMBER', {'timber': 5}]
    METALLIC_ELEMENT_TR = ['METALLIC ELEMENT', {'metallic element':5}]
    METALLIC_ALLOY_TR = ['METALLIC ALLOY', {'metallic alloy':5}]
    ELECTRONICS_TR = ['ELECTRONICS', {'electronics':5}]
    POTENTIAL_FOSSIL_ENERGY_TR = ['POTENTIAL_FOSSIL_ENERGY_TR',{'potential fossil energy':5}]
    POTENTIAL_FOSSIL_USABLE_TR = ['POTENTIAL_FOSSIL_USABLE_TR',{'potential fossil usable':5}]
    RENEWABLE_ENERGY_TR = ['RENEWABLE_ENERGY_TR',{'renewable energy':5}]
    
    #total transfer template list
    ALL_TEMPLATES_TRANSFER = [FOOD_TR,WATER_TR,TIMBER_TR,METALLIC_ELEMENT_TR,METALLIC_ALLOY_TR,ELECTRONICS_TR,
                          POTENTIAL_FOSSIL_ENERGY_TR,POTENTIAL_FOSSIL_USABLE_TR,RENEWABLE_ENERGY_TR]
    
    #define class
    class Node:
        def __init__(self, state_quality, exp_util, sched, countrydf):
            self.state_quality = state_quality
            self.exp_util = exp_util
            self.sched = sched
            self.countrydf = countrydf
        def getSqual(self):
            return self.state_quality
        def getExutil(self):
            return self.exp_util
        def getSched(self):
            return self.sched
        def getCountrydf(self):
            return self.countrydf
        def __lt__(self, other):
            return self.exp_util < other.exp_util
        def __str__(self):
            s = "Node with State Quality: {}\nExpected Utility: {}\nSchedule: {}\nDataframe:\n{}".format(self.state_quality, self.exp_util, self.sched, self.countrydf)
            return s

        
    #define priority queue
    class BoundedPriorityQueue:
        def __init__(self, bound=None):
            self.bound = bound
            self.queue = list()
        def push(self, priority, item):
            if len(self.queue) == self.bound:
                pq.heappushpop(self.queue, (priority, item))
            else:
                pq.heappush(self.queue, (priority, item))
        def pop(self):
            if len(self.queue) >= 1:
                return pq.heappop(self.queue)[-1]
            print("no nodes on fringe")
        def clear(self):
            self.queue.clear()
        def len(self):
            return len(self.queue)
        def getState(self, index):
            if len(self.queue) > index:
                return self.queue[index][1]
            print("index out of bound")
        def pState(self, index):
            if len(self.queue) > index:
                print(self.queue[index][1])
            print("index out of bound")
        def getPriority(self, index):
            if len(self.queue) > index:
                return self.queue[index][0]
            print("index out of bound")
        def popBest(self):
            if len(self.queue) >= 1:
                return pq.nlargest(1, self.queue)[0][1]
            print("no nodes on fringe")
    
    #define root node
    root_node = Node(0, 0, [], country_df)
    successors = generate_successors(root_node, country)
    
    while tree_size > 1:
        for node in successors:
            #search functiom
            tree_size = tree_size - 1